# **Whats an API?**

**API stands for Application Programming Interface**

- "A computing interface which defines interactions between multiple software intermediaries";
basically like the controls of a car.

Examples:

- The thing that retrieves your friend list on Facebook.
- Python modules.
- The connection to your DBMS.

### **REST APIs**

**REST** - **RE**presentional **S**tate **T**ransfer

**Requests must have:**

- URL: Where to go.
- Method: What to do.
- List of Headers: Meta-information and authentication stuff.
- Body: Data to send to the server.

### **Common REST methods:**

- **POST**: Create new resource
- **GET**: Retrieve resource
- **PUT**: Update existing resource
- **DELETE**: Delete existing resource

---

## **Data formats**

Data from an API is usually returned as either JSON or XML.

- **JSON**: JavaScript Object Notation
    - Key-value pairs; like a Python dictionary
    - May be nested; object within objects

    {
        "crust": "original",
        "topping": ["cheese", "pepperoni", "garlic"],
        "status": "cooking"
    }

- **XML**: Extensible Markup Language
    - More verbose than JSON
    - Somewhat trickier to parse with Python

    <order>
        <crust>original</crust>¨
        <toppings>
            <topping>cheese</topping>
            <topping>pepperoni</topping>
            <topping>garlic</topping>
        </topping>
        <status>cooking</status>

### **Overview of HTTP response codes**

- 1xx - Informational response
- 2xx - Success
    - 200: OK
    - 201: Created
    - 204: No content
- 3xx - Redirection
- 4xx - Client errors
    - 400: Bad request
    - 401: Unauthorized to access resource
    - 404: Resource not found
    - 418: I'm a teapot
- 5xx - Server errors

### **Authorisation of web APIs**

Most APIs require some form of authentication

**Token-based**
- Successfully logging in returns a token
- Token is reused for all subsequent requests
- Token may be refreshed
<br>

**API key**
- Permanent or one-time code
- Common for semi-open APIs
<br>

**Token and API are usually included in the HTTP Header**
- For instance X-API-KEY: {key} or Authentication: Bearer {key}
- API-key should not be included in the URL, although some APIs do<br>
(This includes the risk of leaking the key, for instance by leaking logs)

---

# **The Request Module**

**Documentation** - <https://requests.readthedocs.io/en/latest/>

In [ ]:
# Sending a request to the API

import requests

def get_all_public_holidays():
    URL = "http://date.nager.at/api/v3/PublicHolidays/2022/NO"
    response = requests.get(URL)
    if response.ok:
        return response.json()

get_all_public_holidays()


In [ ]:
# We can use an f-string to pass in parameters in the URL, if we want holidays for a 
# spesific country and year:

def get_public_holidays(year, country_code):
    URL = f"http://date.nager.at/api/v3/PublicHolidays/{year}/{country_code}"
    response = requests.get(URL)
    if response.ok:
        return response.json()
    return f"Could not fetch holidays for {country_code} in {year}"

get_public_holidays(2021, "NO")

In [ ]:
# Get the date for a holiday the last x years

from datetime import date

def get_norwegian_holiday_dates(holiday_name, years_back):
    holidays = []
    this_year = date.today().year
    for year in range(this_year, this_year-years_back, -1):
        for holiday_dict in get_public_holidays(year, "NO"):
            if holiday_dict["localName"].lower() == holiday_name.lower():
                holidays.append(holiday_dict["date"])
    return holidays

get_norwegian_holiday_dates("Langfredag", 3)

In [ ]:
from datetime import datetime

def get_date_facts(datestring):
    """ 
    Returns the fact about the given datestring
    We assume a format of 2022-04-15, e.g, YYYY-mm-dd

    """

    dateobj = datetime.strptime(datestring, "%Y-%m-%d")
    fact = query_fact_api(dateobj)
    return {"date": datestring, "fact": fact}

def query_fact_api(dateobj):
    URL = f"http://numbersapi.com/{dateobj.month}/{dateobj.day}/date"
    response = requests.get(URL)
    if response.ok:
        return response.text
    return ""

for holiday in get_norwegian_holiday_dates("Langfredag", 3):
    print(get_date_facts(holiday))